In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

sns.set()

In [ ]:
##### PARAM #####
last_complete_month = '2020-07-31'

In [ ]:
def date_parser(m):
    eng_months = {'januari' : 'January',
                 'februari' : 'February',
                 'mars' : 'March',
                 'april' : 'April',
                 'maj' : 'May',
                 'juni' : 'June',
                 'juli' : 'July',
                 'augusti' : 'August',
                 'september' : 'September',
                 'oktober' : 'October',
                 'november' : 'November',
                 'december' : 'December'}
    
    eng = eng_months[m]
    
    return pd.datetime.strptime(eng,'%B').date().strftime('%B')
    
    
death_df = pd.read_csv('scb_dead_per_month.csv',encoding='ISO-8859-1')

In [ ]:
death_df['månad'] = death_df['månad'].apply(date_parser)
death_df['månad'] = pd.Categorical(death_df['månad'],['January','February','March','April','May','June',
                                         'July','August','September','October','November','December'])

In [ ]:
death_df

In [ ]:
death_df = death_df.groupby('månad').sum()

death_df

In [ ]:
death_df.iloc[:,:].sum()

In [ ]:
scb_pop = pd.read_csv('scb_population.csv',encoding='ISO-8859-1')

scb_pop

In [ ]:
pd.set_option("display.precision", 2)
deaths_per_m = death_df / (scb_pop.loc[0,:] / 1e6)
deaths_per_m = deaths_per_m.T
deaths_per_m

In [ ]:
monthly_deaths = []

for r,v in deaths_per_m.iterrows():
    monthly_deaths.extend(v)
    


In [ ]:
dates = pd.date_range('1990-01-01','2019-12-31',freq='M')
dates

In [ ]:
monthly_timeline = pd.DataFrame(data=monthly_deaths,index=dates,columns=['dead_per_million'])
scb_1990_2019 = monthly_timeline.copy()

In [ ]:
scb_pop.mean()['2015':'2019'].mean()

In [ ]:
curr_pop = 10327589 #from scb.se for 2019-12-31

scb_2020 = pd.read_csv('scb-dead_per_day.csv',sep=';')
scb_2020 = scb_2020.groupby('Månad').sum()
scb_2020['min_2015_2019'] = scb_2020[['2015','2016','2017','2018','2019']].min(axis=1)
scb_2020['max_2015_2019'] = scb_2020[['2015','2016','2017','2018','2019']].max(axis=1)

scb_2020 = scb_2020[['2020','2015-2019','min_2015_2019','max_2015_2019']]
scb_2020_orig = scb_2020.copy()

####
# dividing all years 2015-2020 by 2020 population for consistency with scb_poisson
####

'''
scb_2020[['2015-2019','min_2015_2019',
          'max_2015_2019']] = scb_2020[['2015-2019','min_2015_2019',
            'max_2015_2019']] / (scb_pop.mean()['2015':'2019'].mean()/1e6)
'''

scb_2020[['2015-2019','min_2015_2019',
          'max_2015_2019']] = scb_2020[['2015-2019','min_2015_2019',
            'max_2015_2019']] / (curr_pop/1e6)

scb_2020 = pd.DataFrame(scb_2020)
scb_2020.reset_index(inplace=True)

scb_2020['Månad'] = pd.Categorical(scb_2020['Månad'],['januari','februari','mars','april','maj','juni',
                                                      'juli','augusti','september','oktober','november','december'])


scb_2020.sort_values('Månad',inplace=True)
scb_2020.index = pd.date_range('2020-01-01','2020-12-31',freq='M')
#scb_2020 = scb_2020.loc[:last_complete_month]
scb_2020['2020'] = scb_2020['2020'] / (curr_pop/1e6)
scb_2020.rename(columns={'2020' : 'dead_per_million'},inplace=True)
scb_2020

In [ ]:
scb_2020['dead_per_million'].cumsum() - scb_2020['2015-2019'].cumsum()

In [ ]:
monthly_timeline

In [ ]:
monthly_timeline.loc[:'2019-12-31','dead_per_million'].plot(figsize=(18,12))
plt.plot(scb_2020.loc[:last_complete_month,'dead_per_million'],color='r')

In [ ]:
scb_2020

In [ ]:
monthly_timeline = pd.concat([monthly_timeline['dead_per_million'],scb_2020['dead_per_million']],axis=0)
monthly_timeline

In [ ]:
two_thousands_rank = monthly_timeline.rank(ascending=False).sort_values()
two_thousands_rank = two_thousands_rank.loc['2000-01-01':]
two_thousands_rank.plot(figsize=(18,12),style=['o--'])
plt.ylabel('rank')

In [ ]:
monthly_timeline = pd.DataFrame(monthly_timeline.sort_values(ascending=False))
monthly_timeline['rank'] = range(1,len(monthly_timeline) + 1)
monthly_timeline.head(50)

In [ ]:
mean = monthly_timeline.mean()
mean

In [ ]:
monthly_timeline.loc[last_complete_month]

In [ ]:
top_20 = pd.DataFrame(monthly_timeline[:20])
top_20

In [ ]:
colors = ['b'] * 18 # must set colors after seeing which place they fall onto
colors.append('r')
colors.append('b')
print (colors)
top_20.index = top_20.index.date
top_20['dead_per_million'].plot(kind='bar',
                                figsize=(18,12),
                                color=colors,
                                title='SWEDEN - Top-20 deadliest months January 1990 - May 2020\nDataSource: SCB.se')
plt.ylabel('deaths per million')
plt.axhline(mean['dead_per_million'],color='orange',ls='dashed',label='average 1990-2019')
plt.legend(loc='upper right')
plt.savefig('scb_population_deaths_1990-2020.jpg',format='jpg')

In [ ]:

scb_1990_2019

In [ ]:
scb_1990_2020 = pd.DataFrame(pd.concat([scb_1990_2019['dead_per_million'],scb_2020['dead_per_million']],axis=0))
scb_1990_2020

In [ ]:
scb_1990_2020['year'] = scb_1990_2020.index.year
scb_1990_2020['month'] = scb_1990_2020.index.month

In [ ]:
scb_1990_2020.tail(20)

In [ ]:
month_grp = scb_1990_2020[:'2019-12-31'].groupby('month')['dead_per_million'].agg(['min','mean','max'])
print (month_grp.head(12))
scb_2020


In [ ]:
scb_2020['excess'] = scb_2020['dead_per_million'] - scb_2020['2015-2019']
scb_2020

In [ ]:
excess = scb_2020.set_index(scb_2020.index.month).join(month_grp['mean'])
excess.drop(['min_2015_2019','max_2015_2019'],axis=1,inplace=True)


In [ ]:
excess.rename(columns={'mean' : '1990-2019_mean'}, inplace=True)
excess['excess_1990_baseline'] = excess['dead_per_million'] - excess['1990-2019_mean']
excess['excess_2015_baseline'] = excess['dead_per_million'] - excess['2015-2019']
excess['cumulative_excess_vs_1990-2019'] = excess['excess_1990_baseline'].cumsum()
excess['cumulative_excess_vs_2015-2019'] = excess['excess_2015_baseline'].cumsum()
excess['date'] = pd.date_range('2020-01-01','2020-12-31',freq='M')

excess[:pd.Timestamp(last_complete_month).date().month].plot(kind='bar',figsize=(18,12),
                                                             title=('Sweden Excess Deaths 2020, baselines 1990-2019 & 2015-2019'),
                                                             y=['cumulative_excess_vs_1990-2019',
                                                               'cumulative_excess_vs_2015-2019'],color=['orange','red'])
plt.ylabel('Excess Deaths per Million')
plt.xlabel('Month')
excess

In [ ]:
foo_2020 = pd.read_csv('scb-dead_per_day.csv',sep=';')
foo_2020.drop(['DagMånad','-','Sort','Unnamed: 12','Unnamed: 13','Dag'],axis=1,inplace=True)
foo_2020['Månad'] = foo_2020['Månad'].apply(date_parser)
foo_2020['Månad'] = pd.Categorical(foo_2020['Månad'],['January','February','March','April','May','June',
                                         'July','August','September','October','November','December'])
foo_2020 = foo_2020.groupby('Månad').sum()
foo_2020['excess'] = foo_2020['2020'] - foo_2020['2015-2019']
foo_2020['excess_per_M'] = foo_2020['excess'] /(curr_pop / 1e6)
foo_2020['2020_cum'] = foo_2020['2020'].cumsum()
foo_2020['2015-2019_cum'] = foo_2020['2015-2019'].cumsum()
foo_2020['excess_cum'] = foo_2020['2020_cum'] - foo_2020['2015-2019_cum']
foo_2020.drop(['2015','2016','2017','2018','2019'],inplace=True,axis=1)
foo_2020['2020_per_M'] = foo_2020['2020'] / (curr_pop /1e6)
foo_2020['2015-2019_per_M'] = foo_2020['2015-2019'] / (curr_pop / 1e6)
foo_2020['2020_per_M_cum'] = foo_2020['2020_per_M'].cumsum()
foo_2020['2015-2019_per_M_cum'] = foo_2020['2015-2019_per_M'].cumsum()
foo_2020['excess_per_M_cum'] = foo_2020['2020_per_M_cum'] - foo_2020['2015-2019_per_M_cum']
foo_2020


In [ ]:
scb_2020['month'] = scb_2020.index.month 

plt.figure(figsize=(18,12))
plt.title('Sweden Monthly Deaths per million, adjusted for population - All Causes\nDataSource:SCB.se')
ax = plt.gca()
ax2 = plt.twinx()

ax.plot(month_grp['mean'],'o--',label='Monthly Average 1990-2019')
#ax.plot(scb_2020['month'],scb_2020['2015-2019'],'o--',color='k',label='Monthly Average 2015-2019')
#plt.fill_between(scb_2020['month'],scb_2020['min_2015_2019'],scb_2020['max_2015_2019'],color='m',alpha=0.3,label='2015-2019 range')

ax.fill_between(month_grp.index,month_grp['min'],month_grp['max'],color='c',alpha=0.3,label='1990-2019 range')

ax.plot(scb_2020['month'][:last_complete_month],scb_2020['dead_per_million'][:last_complete_month],'o--',color='crimson',label='2020')
ax.legend(loc='upper left')
ax.set_xlabel('month')
ax.set_ylabel('dead per million')

#ax2.plot(excess['cum_vs_2015'][:pd.Timestamp(last_complete_month).month],'o--',
         #color='orange',label='Excess deaths cmp 2015-2019')

ax2.plot(excess['cumulative_excess_vs_1990-2019'][:pd.Timestamp(last_complete_month).month],'o--',
         color='orange',label='Excess deaths cmp 1990-2019\nNOTE!! AXIS VALUES ARE NEGATIVE!')

ax2.set_ylabel('Cumulative Excess Deaths per Million')
#ax2.set_ylim([-400,10])
ax2.legend(loc='upper right')

xticks = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
_=plt.xticks(month_grp.index,xticks)
plt.savefig('scb_monthly_deaths_all_cmp_1990_2019.jpg',format='jpg')

In [ ]:
plt.figure(figsize=(18,12))
plt.title('Sweden Monthly Deaths per million, adjusted for population - All Causes\nDataSource:SCB.se')
ax = plt.gca()
ax2 = plt.twinx()

#ax.plot(month_grp['mean'],'o--',label='Monthly Average 1990-2019')
ax.plot(scb_2020['month'],scb_2020['2015-2019'],'o--',color='b',label='Monthly Average 2015-2019')
ax.fill_between(scb_2020['month'],scb_2020['min_2015_2019'],scb_2020['max_2015_2019'],color='c',alpha=0.3,label='2015-2019 range')

#ax.fill_between(month_grp.index,month_grp['min'],month_grp['max'],color='c',alpha=0.3,label='1990-2019 range')

ax.plot(scb_2020['month'][:last_complete_month],scb_2020['dead_per_million'][:last_complete_month],'o--',color='crimson',label='2020')
ax.legend(loc='upper left')
ax.set_xlabel('month')
ax.set_ylabel('dead per million')

#ax2.plot(excess['cum_vs_2015'][:pd.Timestamp(last_complete_month).month],'o--',
         #color='orange',label='Excess deaths cmp 2015-2019')

ax2.plot(excess['cumulative_excess_vs_2015-2019'][:pd.Timestamp(last_complete_month).month],'o--',
         color='orange',label='Excess deaths cmp 2015-2019')

ax2.set_ylabel('Cumulative Excess Deaths per Million')
ax2.legend(loc='upper right')

xticks = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
_=plt.xticks(month_grp.index,xticks)
plt.savefig('scb_monthly_deaths_all_cmp_2015-2019.jpg',format='jpg')

In [ ]:
scb_2020_monthly = scb_2020.set_index('month')
print (scb_2020_monthly)

mean_1990_2019 = month_grp.loc[:pd.Timestamp(last_complete_month).month]
print (mean_1990_2019)
monthly_excess = scb_2020_monthly['dead_per_million'] - mean_1990_2019['mean']
print (monthly_excess)
print (monthly_excess.cumsum())

In [ ]:
pd.Timestamp(last_complete_month).month

In [ ]:
mask = scb_1990_2020.index.month.isin(range(1,pd.Timestamp(last_complete_month).month + 1))
first_months = scb_1990_2020[mask]['dead_per_million']
first_cum = first_months.groupby(first_months.index.year).sum()
first_cum

In [ ]:
first_cum.plot(kind='bar',figsize=(18,12),color='orange',
               title='SWEDEN cumulative deaths per million Jan 1:st - {} last,1990-2020\nDataSource:SCB.se'.format(
                   pd.Timestamp(last_complete_month).month_name()))
plt.ylabel('deaths per million')
plt.savefig('scb_deaths_jan_may_unsorted.jpg'.format('jpg'))

In [ ]:
colors = ['orange'] * 23
colors.append('crimson')
color_tail = ['orange'] * 7
colors.extend(color_tail)

print (colors)
first_cum.sort_values(ascending=False).plot(kind='bar',figsize=(18,12),
                                            title='SWEDEN cumulative deaths per million Jan 1:st - {} last,1990-2020\nDataSource:SCB.se'.format(
                   pd.Timestamp(last_complete_month).month_name()),
                                            color=colors)
plt.ylabel('deaths per million')
plt.savefig('scb_deaths_jan_may_sorted.jpg',format='jpg')

In [ ]:
by_month = first_months.groupby(first_months.index.month)
month_stats =[]

for month in range (1,pd.Timestamp(last_complete_month).month + 1):
    m = by_month.get_group(month)
    month_stats.append(m.agg(['min','mean','max']).values)
    
month_stats = pd.DataFrame(data=np.array(month_stats),
                           index=range(1,pd.Timestamp(last_complete_month).month + 1),
                           columns=['mini','mean','maxi'])

month_stats.index.name='month'
month_stats

In [ ]:
from matplotlib import cm

first_cumsum = first_months.groupby(first_months.index.year).cumsum()
bar = first_cumsum.groupby(first_cumsum.index.year)

plt.figure(figsize=(18,12))
plt.title('SWEDEN cumulative monthly deaths per million, 1990-2020, Jan 1st to {} last'.format(
    pd.Timestamp(last_complete_month).month_name()))

for k in bar.groups.keys():
    plt.plot(bar.get_group(k),'ro--')
    
plt.ylabel('cumulative deaths per million')

In [ ]:
first_months

In [ ]:
first_sum = first_months.groupby(first_months.index.year)

month_names = {1:'Jan',2:'Feb',3:'Mar',4:'Apr',5:'May',6:'Jun',
              7:'Jul',8:'Aug',9:'Sep',10:'Oct',11:'Nov',12:'Dec'}

years = dict()

for k in first_sum.groups.keys():
    years[k] = first_sum.get_group(k).values
    
years = pd.DataFrame(years).T
num_months = range(1,pd.Timestamp(last_complete_month).month + 1)
years.columns= num_months

years.rename(columns=month_names,inplace=True)

years.columns.name = 'month'
years.index.name = 'year'
years_per_M = years



In [ ]:
years_1990_2019_mean = years.loc[1990:2019].mean().sum()
years_2015_2019_mean = years.loc[2015:2019].mean().sum()
print (years_1990_2019_mean)
print (years_2015_2019_mean)
years

In [ ]:


years.plot(kind='bar',stacked=True,figsize=(18,12),
          title='Sweden1990-2020 : Deaths per million : All Causes : January to {}\nDataSource:SCB.se'.format(
              pd.Timestamp(last_complete_month).month_name()))

plt.axhline(years_1990_2019_mean,ls='dashed',color='crimson',label='1990-2019 average')
plt.axhline(years_2015_2019_mean,ls='dashed',color='orange',label='2015-2019 average')

plt.ylabel('Nr dead per million')

plt.legend(framealpha=0.1, loc='best')
plt.savefig('scb_deaths_per_M_per_month_1990_2020.jpg',format='jpg')

In [ ]:
plt.figure(figsize=(18,12))
for month in years.columns:
    
    plt.plot(years.index,years[month],'o--',label=month)
    
plt.legend(loc='upper right')


In [ ]:
scatter_scale_factor = 100
#years = years[:10]
years = years / scatter_scale_factor

colors =['r','b','g','yellow','orange'] * 6
colors.append('crimson')

plt.figure(figsize=(18,12))
plt.title('Sweden : Deaths per 10.000 January - {} 1990 - 2020\nDataSource:SCB.se'.format(
    pd.Timestamp(last_complete_month).month_name()))
plt.grid(True)

for r in range(len(years.index)):
    for c in range(len(years.columns) ):
        ndots = int(np.round(years.iloc[r,c]))
        
        
        for i in range (0,ndots):
            rr = 0.1 + r + 0.8 * np.random.random()
            cc = 0.1 + c + 0.8 * np.random.random()
            
            plt.scatter(cc,rr,color = colors[r])

yticks = range(1990,2022)

random_labels = list('ABCDEFGHIJKLMNOPQRSTUVXYZ')
random_labels.extend(('AA','BB','CC','DD','EE','FF'))
#yticks = random_labels

plt.yticks(range(len(years) + 1),yticks)

month_names = ['Jan','Feb','Mar','Apr','May','Jun']
plt.xticks(range(len(years.columns) + 1),month_names)

plt.ylabel('year')
plt.xlabel('month')
print (years)
plt.savefig('scb_1990_2019_deaths_scatter.jpg',format='jpg')


In [ ]:
print (years)
deaths_per_month = years.sum(axis=0)


deaths_per_year = years.sum(axis=1) 

print (deaths_per_month)
print (deaths_per_year)

In [ ]:
deaths_per_year.plot(kind='barh',title='Sweden deaths per 10.000 Jan - {} 1990-2020'.format(
    pd.Timestamp(last_complete_month).month_name()),color='orange',
                     figsize=(18,12))

plt.xlabel('deaths per 10.000')
plt.ylabel('year')

In [ ]:
years = years_per_M
print (years[:-1])

average_2015_2019 = years.loc[2015:2019].mean()
print (average_2015_2019)
monthly_average = years[:-1].mean()
print (monthly_average)
monthly_percentiles = years[:-1].quantile(1,axis=0)
monthly_percentiles


In [ ]:
fhm_dead = pd.read_csv('fhm_scb_dead.csv',sep=';',header=0,index_col=0,parse_dates=True,names=['fhm_dead'])
fhm_dead


In [ ]:
fhm_monthly = fhm_dead.groupby(fhm_dead.index.month).sum()
fhm_monthly['fhm_dead_per_M'] = fhm_monthly['fhm_dead'] / (curr_pop / 1e6)
fhm_monthly.at[1,:] = [0,0]
fhm_monthly.at[2,:] = [0,0]
fhm_monthly.sort_index(inplace=True)
fhm_monthly

In [ ]:
scb_dead_2020_per_M = pd.DataFrame(years.loc[2020].copy())
scb_dead_2020_per_M['month_nr'] = range(1,len(scb_dead_2020_per_M) + 1)
scb_dead_2020_per_M.reset_index(inplace=True)
scb_dead_2020_per_M.set_index('month_nr',inplace=True)
scb_dead_2020_per_M

In [ ]:
all_deaths = scb_dead_2020_per_M.join(fhm_monthly)
all_deaths.rename(columns={2020 : 'scb_dead_per_M',
                          'fhm_dead_per_M' : 'covid_deaths_per_M'},inplace=True)
all_deaths['other_deaths_per_M'] = all_deaths['scb_dead_per_M'] - all_deaths['covid_deaths_per_M']
all_deaths['pct_covid_deaths'] = all_deaths['covid_deaths_per_M'] / all_deaths['scb_dead_per_M'] * 100
all_deaths['pct_other_deaths'] = all_deaths['other_deaths_per_M'] / all_deaths['scb_dead_per_M'] * 100
all_deaths

In [ ]:
all_deaths.plot(title='SWEDEN COVID-19 deaths vs OTHER deaths Jan - {} 2020\nDataSource: SCB.se, FHM'.format(
    pd.Timestamp(last_complete_month).month_name()),
                kind='bar',x='month',color=['r','orange'],
                y=['covid_deaths_per_M','other_deaths_per_M'],stacked=True,figsize=(18,12))

plt.ylabel('Total deaths per million')
plt.plot(monthly_average.index,monthly_average,'o--',label='monthly average 1990-2019',color='red')
plt.plot(average_2015_2019.index,average_2015_2019,'o--',label='monthly average 2015-2019',color='k')

plt.legend(loc='upper right')
plt.savefig('scb_fhm_covid_vs_other_abs.jpg',format='jpg')

In [ ]:
average_2015_2019

In [ ]:
all_deaths.plot(title='SWEDEN : Percent COVID-19 deaths vs OTHER deaths Jan - {} 2020\nDataSource: SCB.se, FHM'.format(
    pd.Timestamp(last_complete_month).month_name()),
                kind='bar',x='month',color=['r','orange'],
                y=['pct_covid_deaths','pct_other_deaths'],stacked=True,figsize=(18,12))

plt.ylabel('Percent')
plt.savefig('scb_fhm_covid_vs_other_rel.jpg',format='jpg')

In [ ]:
print (years.min())
print (years.max())
print (years)


In [ ]:
years.sum(axis=1).sort_values(ascending=False)

In [ ]:
cum_mean_2015_2019 = years.sum(axis=1)
cum_mean_2015_2019 = cum_mean_2015_2019.loc[2015:2020].mean()
print (cum_mean_2015_2019)

cum_mean_1990_2019 = years.sum(axis=1)
cum_mean_1990_2019 = cum_mean_1990_2019.loc[1990:2020].mean()
print (cum_mean_1990_2019)

print (years.loc[2020].sum())

In [ ]:
scb_2020_copy = scb_2020_orig.copy()
scb_2020_copy.reset_index(inplace=True)
#scb_2020_copy['Månad'] = scb_2020_copy['Månad'].apply(date_parser)


scb_2020_copy['Månad'] = pd.Categorical(scb_2020_copy['Månad'],['januari','februari','mars','april','maj','juni',
                                                      'juli','augusti','september','oktober','november','december'])

scb_2020_copy.sort_values('Månad',inplace=True)
scb_2020_copy.drop(['min_2015_2019','max_2015_2019'],axis=1,inplace=True)

scb_2020_copy['excess_2015'] = scb_2020_copy['2020'] - scb_2020_copy['2015-2019']
scb_2020_copy['excess_2015_cum'] = scb_2020_copy['excess_2015'].cumsum()
scb_2020_copy['2020_cum'] = scb_2020_copy['2020'].cumsum()
scb_2020_copy['2020_per_M'] = scb_2020_copy['2020'] / (curr_pop / 1e6)
scb_2020_copy['2020_per_M_cum'] = scb_2020_copy['2020_per_M'].cumsum()
scb_2020_copy['excess_2015_per_M'] = scb_2020_copy['excess_2015'] / (curr_pop / 1e6)
scb_2020_copy['excess_2015_per_M_cum'] = scb_2020_copy['excess_2015_per_M'].cumsum()
scb_2020_copy

In [ ]:
mean_1990_2019 = scb_1990_2019.groupby(scb_1990_2019.index.month).mean()
mean_1990_2019.index = scb_2020_copy.index

scb_2020_copy['1990-2019'] = mean_1990_2019
mean_1990_2019

In [ ]:
scb_2020_copy['excess_1990_per_M'] = scb_2020_copy['2020_per_M'] - scb_2020_copy['1990-2019']
scb_2020_copy['excess_1990_per_M_cum'] = scb_2020_copy['excess_1990_per_M'].cumsum()
scb_2020_copy.set_index('Månad',inplace=True)
scb_2020_copy

In [ ]:
scb_2020_copy[['excess_2015_per_M_cum',
               'excess_1990_per_M_cum']][:'juli'].plot(
    kind='bar',color=['r','orange'],
    title='Sweden Cumulative Excess Death 2020 Y2D, baseline: averages 2015-2019 & 1990-2019\nDataSource:SCB.se',
figsize=(18,12))

plt.ylabel('Cumulative Excess deaths per million')
plt.legend(['cmp to 2015-2019 average','cmp to 1990-2019 average'],loc='upper left')
plt.savefig('excess_deaths_cumulative.jpg',format='jpg',dpi='figure')